In [1]:
import pandas as pd
import nltk

In [2]:
#libreria tokenizador
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mcm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#libreria palabras vacias
from nltk.corpus import stopwords
nltk.download("stopwords")
palabras_vacias=set(stopwords.words("spanish"))
#print(palabras_vacias)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mcm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Libreria lematizador install para jupyterLab
#!pip install spacy

In [ ]:
#!python -m spacy download es_core_news_sm

In [4]:
import es_core_news_sm
import spacy
lematiza = es_core_news_sm.load()

In [5]:
#Steamming
from nltk import SnowballStemmer

In [6]:
df_MultiL=pd.read_csv('MultiL.csv', encoding='utf-8')
print(df_MultiL.shape[0])

270399


In [7]:
#filtra por lenguaje español
df_Spanish=df_MultiL.query("language=='spanish'")
df_Spanish = df_Spanish.drop('source', axis=1)
df_Spanish = df_Spanish.drop('language', axis=1)
df_Spanish = df_Spanish.reset_index(drop=True)
textos=df_Spanish['text']
print(df_Spanish.shape[0])

1839


In [12]:
pd.set_option('display.max_colwidth', -1)
df_Spanish.head(20)

C:\Users\mcm\AppData\Local\Temp\ipykernel_11308\3127228529.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,text,label
0,estoy hasta el ojete de que me digáis que tengo cara de mala leche,negative
1,@user Por? Tenía pensado verla después de la segunda de Daredevil,neutral
2,Esto de estar feliz mola,positive
3,Ya no es tan divertido,negative
4,"@user te recuerdo que soy una persona que tiene criterio, equivocado, pero lo tengo",neutral
5,@user @user @user con una pequeña donación hará felices a miles de chicas que no tienen #asociacionmariloli,positive
6,He probado una nueva espuma para el pelo y sí que lo deja más rizado pero se queda como efecto gomina y no me gusta.,negative
7,@user Ojalá pudiera darte el abrazo en vivo o al menos acompañarte hoy. Siento mucho lo de tu primo,neutral
8,@user aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español. Os llevareis bien @user,positive
9,"@user ya somos dos, que triste",negative


In [17]:
df_True=df_Spanish.query("label=='positive'")
df_False=df_Spanish.query("label=='negative'")
df_Neutro=df_Spanish.query("label=='neutral'")

print(df_True.shape[0])
print(df_False.shape[0])
print(df_Neutro.shape[0])

613
613
613


In [18]:
import nltk
from nltk import FreqDist

# Obtener la frecuencia de las palabras en el DataFrame
frecuencia_palabras = FreqDist(df_Spanish['text'])

# Establecer un umbral mínimo de frecuencia
umbral_minimo = 3

# Identificar palabras de baja frecuencia
palabras_baja_frecuencia = [palabra for palabra, frecuencia in frecuencia_palabras.items() if frecuencia < umbral_minimo]

# Función para eliminar palabras de baja frecuencia en un texto
def eliminar_baja_frecuencia(texto):
    palabras = texto.split()
    palabras_sin_baja_frecuencia = [palabra for palabra in palabras if palabra not in palabras_baja_frecuencia]
    texto_sin_baja_frecuencia = ' '.join(palabras_sin_baja_frecuencia)
    return texto_sin_baja_frecuencia

# Aplicar la función a la columna de texto en el DataFrame original
df_Spanish['text'] = df_Spanish['text'].apply(eliminar_baja_frecuencia)

# Crear un nuevo DataFrame con las columnas deseadas
#df_Spanish = df_Spanish[['texto_sin_baja_frecuencia', 'otra_columna']]

# Imprimir el nuevo DataFrame
#print(df_Spanish.head())

In [19]:
#dataframes donde se almacena lema y stemmer
df_lema=pd.DataFrame({'texto': []})
df_Stemmer=pd.DataFrame({'texto': []})
df_limpieza=pd.DataFrame({'texto': []})

In [58]:
lineaLeida=0
cammel=0
log=500
Listalema = []
cadenalema=''

df_lema = df_lema.drop(index=df_lema.index)
df_Stemmer= df_Stemmer.drop(index=df_Stemmer.index)
df_limpieza= df_Stemmer.drop(index=df_Stemmer.index)

for texto in textos:
    lineaLeida+=1
    if(lineaLeida<=log):
        print("---------------------------------- Linea "+ str(lineaLeida) +" ---------------------------------- ")
        #eliminar menciones de @user
        print('--Elimina @user')
    texto=re.sub('@user',"",texto)
    #Eliminar caracteres especiales: + / ? ! ¡ ...
    if(lineaLeida<=log):
        print('--Elimina caracteres especiales [,\.$\*¿\?¡!\+\-\[\]\)\(\\\]"," ')
        
        
    #-------------------------------------------------------------------------------------------------------------------------------------    
    texto = re.sub(r'\b\w\b', '', texto)#------------------------------------------------->retira texto de una sola palabra
    texto = re.sub(r'\b(\w)\1+\b', '', texto)#-------------------------------------------->retira texto de una sola palabra repetida
    texto = re.sub(r'[^\w\s]', '', texto)#------------------------------------------------>retira emoticones 
    texto = re.sub(r'http\S+|www\S+', '', texto)#----------------------------------------->retira URL's
    #texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)#----------------------------------------->elimina caracteres especiales, deja espacios en blanco
    texto=re.sub(r' {2,}', ' ', texto)#--------------------------------------------------->retira multiples espacios en blaco, deja solo 1
    #-------------------------------------------------------------------------------------------------------------------------------------  
    
        
    texto=re.sub("[,\d+\.$\*¿\?¡!\+\-\[\]\)\(\\\]","",texto)#--->quita numeros y comas
    #texto=re.sub("[,\.$\*¿\?¡!\+\-\[\]\)\(\\\]","",texto)
    
    #Para los hashtags, solo eliminar el símbolo (#)
    if(lineaLeida<=log):
        print('--Retira hashtags')
    texto=re.sub("[#]","",texto)
    #Eliminar correos electrónicos.
    if(lineaLeida<=log):
        print('--Elimina correos electronicos')
    texto= re.sub("(\S+@\S+\.\S)", "", texto)
    
    
    # Separación de palabras en notación Camell: “QuedateEnCasa→ Quedate En Casa”.
    if(lineaLeida<=log):
        print('-- Separa palabras con notacion Camell')
    patron=re.compile('([ÁÉÍÓÚA-Z]*[a-z0-9]+[ÁÉÍÓÚA-Z]+)+')
    for palabra in texto.split(' '):
        if patron.match(palabra)!=None:
            palabraSep = " ".join(re.findall('[ÁÉÍÓÚA-Z]+[a-z0-9]*',palabra))
            texto=re.sub(palabra,palabraSep,texto)
            if(lineaLeida<=log):
                print('       ---- Palabra original: ' + palabra)
                print('       ---- Palabra encontrada*: ' + palabraSep)
            cammel=+1
    #if(cammel<log):
        #if(lineaLeida<=log):
            #print('       ---- Ninguna palabra encontrada')
    cammel=0
    
    
    #Extraer/listar todas las frases que inicien con la preposición “en”, seguido de espacio en blanco y luego una palabra que inicie en Mayúsculas.
    #if(lineaLeida<=log):
        #print('-- Lista frases que inicias con "en" ')
        #print('       ---- Frase encontrada' + str(re.findall('\sen\s[A-Z]\S+',texto)))
    
    #Normalizar todo a minúsculas.
    if(lineaLeida<=log):
        print('-- Normalizar a minusculas')
    
    texto = texto.lower()
    
    if(lineaLeida<=log):
        print('       ---- '+ texto)
    
    #Eliminar palabras vacías.
    if(lineaLeida<=log):
        print('-- Elimina palabras vacías')
        
    texto=" ".join([palabra for palabra in texto.split(' ') if palabra not in palabras_vacias])

    #Eliminar palabras vacías.
    if(lineaLeida<=log):
        print('       ---- '+ texto)

    #retira espacios en blanco al inicio y al final de la oración
    texto=texto.strip()
    texto = re.sub(r'[^a-zá-ú0-9\s]', '', texto)#------------------------------>elimina caracteres especiales, deja espacios en blanco
    
    
    
    if(lineaLeida<=log):
        print('       ---- '+ texto)
    #------------------------------------------------------------
    textoLimpio=texto
    df_nuevafila = pd.DataFrame({'texto': [textoLimpio]})
    df_limpieza = pd.concat([df_limpieza, df_nuevafila], ignore_index=True)
    textoLimpio=''
    #------------------------------------------------------------
    
    #Lematizar
    if(lineaLeida<=log):
        print('-- Lematiza')
    
    textol=lematiza(texto)
    
    
    for palabra in textol:
        lema = palabra.lemma_
        palabra_original = palabra.text
        Listalema.append(lema)  # Agregar el lema a la lista
        if(lineaLeida<=log):
            print('       ---- '+palabra_original+'-->'+lema)
            
    cadenalema = ','.join(Listalema)  #-----------------------------> Unir los lemas con comas
    # Limpiar la lista Listalema
    Listalema.clear()    

    if(lineaLeida<=log):
        print('       Resultado-------->'+cadenalema)
    
    #------------------------------------------------------------
    df_nuevafila = pd.DataFrame({'texto': [cadenalema]})
    df_lema = pd.concat([df_lema, df_nuevafila], ignore_index=True)
    cadenalema=''
    #------------------------------------------------------------
            
    #Stemmer
    if(lineaLeida<=log):
        print('-- Stemmer - Raíz de la palabra')
    s=word_tokenize(texto)
    esp=SnowballStemmer('spanish')
    text_r=[esp.stem(token) for token in s]
    str_text_r = ','.join(text_r)  #-----------------------------> Unir los stemmer con comas  
        
    if(lineaLeida<=log):
        print('       Resultado-------->'+str(str_text_r))
    
    #------------------------------------------------------------
    df_nuevafila = pd.DataFrame({'texto': [str_text_r]})
    df_Stemmer = pd.concat([df_Stemmer, df_nuevafila], ignore_index=True)
    str_text_r=''
    #------------------------------------------------------------
    
    
    

    
    
        
print("Se procesaron: " + str(lineaLeida))

---------------------------------- Linea 1 ---------------------------------- 
--Elimina @user
--Elimina caracteres especiales [,\.$\*¿\?¡!\+\-\[\]\)\(\\]"," 
--Retira hashtags
--Elimina correos electronicos
-- Separa palabras con notacion Camell
-- Normalizar a minusculas
       ---- estoy hasta el ojete de que me digáis que tengo cara de mala leche
-- Elimina palabras vacías
       ---- ojete digáis cara mala leche
       ---- ojete digáis cara mala leche
-- Lematiza
       ---- ojete-->ojete
       ---- digáis-->digáis
       ---- cara-->cara
       ---- mala-->malo
       ---- leche-->leche
       Resultado-------->ojete,digáis,cara,malo,leche
-- Stemmer - Raíz de la palabra
       Resultado-------->ojet,dig,car,mal,lech
---------------------------------- Linea 2 ---------------------------------- 
--Elimina @user
--Elimina caracteres especiales [,\.$\*¿\?¡!\+\-\[\]\)\(\\]"," 
--Retira hashtags
--Elimina correos electronicos
-- Separa palabras con notacion Camell
-- Normalizar a mi

In [33]:
#conteo dataframes
print(df_lema.size)
print(df_Stemmer.size)

1839
1839


In [34]:
pd.set_option('display.max_colwidth', None)
#df_limpieza.to_csv('df_limpieza.csv',index=False)
df_limpieza.head()

,texto
0,ojete digáis cara mala leche
1,pensado verla después segunda daredevil
2,feliz mola
3,tan divertido
4,recuerdo persona criterio equivocado


In [35]:
pd.set_option('display.max_colwidth', None)
df_lema.head()

,texto
0,"ojete,digáis,cara,malo,leche"
1,"pensado,ver él,después,segundo,daredevil"
2,"feliz,mola"
3,"tanto,divertido"
4,"recordar,persona,criterio,equivocado"


In [36]:
df_Stemmer.head()

,texto
0,"ojet,dig,car,mal,lech"
1,"pens,verl,despues,segund,daredevil"
2,"feliz,mol"
3,"tan,divert"
4,"recuerd,person,criteri,equivoc"


In [37]:
#vaciar dataframes
#df_lema = df_lema.drop(index=df_lema.index)
#df_Stemmer= df_Stemmer.drop(index=df_Stemmer.index)

In [ ]:
#pip install -U scikit-learn

In [27]:
#libreria tf-idf de sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
# Crear un objeto TfidfVectorizer
vectorizerLema = TfidfVectorizer()
vectorizerStem = TfidfVectorizer()

# Ajustar el vectorizador en los datos de entrada
vectorizerLema.fit(df_lema['texto'])
vectorizerStem.fit(df_Stemmer['texto'])

# Transformar los datos de entrada en la matriz TF-IDF
tfidf_matrixLema = vectorizerLema.transform(df_lema['texto'])
tfidf_matrixStem = vectorizerStem.transform(df_Stemmer['texto'])

In [39]:
# Obtener el diccionario de términos del vectorizador
terms_dictLema = vectorizerLema.vocabulary_
terms_dictStem = vectorizerStem.vocabulary_

# Obtener los nombres de los términos a partir del diccionario de términos
feature_namesLema = [term for term, index in sorted(terms_dictLema.items(), key=lambda x: x[1])]
feature_namesStem = [term for term, index in sorted(terms_dictStem.items(), key=lambda x: x[1])]

# Convertir la matriz de TF-IDF en un DataFrame de pandas
tfidf_dfLema = pd.DataFrame(tfidf_matrixLema.todense(), columns=feature_namesLema)
tfidf_dfStem = pd.DataFrame(tfidf_matrixStem.todense(), columns=feature_namesStem)

In [40]:
tfidf_dfLema.head(3)

aaahm  aah  aahhh  aandaluciar  abandona  abandonadisimo  abandonado  \
0    0.0  0.0    0.0          0.0       0.0             0.0         0.0   
1    0.0  0.0    0.0          0.0       0.0             0.0         0.0   
2    0.0  0.0    0.0          0.0       0.0             0.0         0.0   

   abandonar  abasto  abceso  abedul  abel  abierto  abogado  abracito  \
0        0.0     0.0     0.0     0.0   0.0      0.0      0.0       0.0   
1        0.0     0.0     0.0     0.0   0.0      0.0      0.0       0.0   
2        0.0     0.0     0.0     0.0   0.0      0.0      0.0       0.0   

   abrazarte  abrazo  abrazote  abrazoteee  abrigo  abrir  abristir  \
0        0.0     0.0       0.0         0.0     0.0    0.0       0.0   
1        0.0     0.0       0.0         0.0     0.0    0.0       0.0   
2        0.0     0.0       0.0         0.0     0.0    0.0       0.0   

   abrumador  abs  absolutamente  absoluto  abstracto  absurdo  abu  abuelito  \
0        0.0  0.0            0.0       0.0        0.0      0.0  0.0       0.0   
1        0.0  0.0            0.0       0.0        0.0      0.0  0.0       0.0   
2        0.0  0.0            0.0       0.0        0.0      0.0  0.0       0.0   

   abuelo  aburrida  aburrido  aburrimiento  aburrir  acaba  acabado  acabar  \
0     0.0       0.0       0.0           0.0      0.0    0.0      0.0     0.0   
1     0.0       0.0       0.0           0.0      0.0    0.0      0.0     0.0   
2     0.0       0.0       0.0           0.0      0.0    0.0      0.0     0.0   

   acabo  acabé  acampar  acc  accederia  accidente  acción  aceit  aceituna  \
0    0.0    0.0      0.0  0.0        0.0        0.0     0.0    0.0       0.0   
1    0.0    0.0      0.0  0.0        0.0        0.0     0.0    0.0       0.0   
2    0.0    0.0      0.0  0.0        0.0        0.0     0.0    0.0       0.0   

   acento  aceptado  aceptar  acerca  acercar  acertado  acertar  achucharte  \
0     0.0       0.0      0.0     0.0      0.0       0.0      0.0         0.0   
1     0.0       0.0      0.0     0.0      0.0       0.0      0.0         0.0   
2     0.0       0.0      0.0     0.0      0.0       0.0      0.0         0.0   

   acierto  aclarar  aclare  acomodado  acompañado  acompañar  acompañarte  \
0      0.0      0.0     0.0        0.0         0.0        0.0          0.0   
1      0.0      0.0     0.0        0.0         0.0        0.0          0.0   
2      0.0      0.0     0.0        0.0         0.0        0.0          0.0   

   acompaño  acondionado  aconsejar  acordar  acoso  acostado  acostumbrado  \
0       0.0          0.0        0.0      0.0    0.0       0.0           0.0   
1       0.0          0.0        0.0      0.0    0.0       0.0           0.0   
2       0.0          0.0        0.0      0.0    0.0       0.0           0.0   

   acostumbrar  acostumbrarme  acostumbré  actitud  activado  actividad  \
0          0.0            0.0         0.0      0.0       0.0        0.0   
1          0.0            0.0         0.0      0.0       0.0        0.0   
2          0.0            0.0         0.0      0.0       0.0        0.0   

   activista  activo  acto  actor  actriz  actuación  actualizacion  \
0        0.0     0.0   0.0    0.0     0.0        0.0            0.0   
1        0.0     0.0   0.0    0.0     0.0        0.0            0.0   
2        0.0     0.0   0.0    0.0     0.0        0.0            0.0   

   actualizado  actualizar  actualizo  acuarelastmr  acuerda  acuerdo  \
0          0.0         0.0        0.0           0.0      0.0      0.0   
1          0.0         0.0        0.0           0.0      0.0      0.0   
2          0.0         0.0        0.0           0.0      0.0      0.0   

   acuerdoo  acuesto  acá  adaptación  adaptado  adc  adelante  adelgazado  \
0       0.0      0.0  0.0         0.0       0.0  0.0       0.0         0.0   
1       0.0      0.0  0.0         0.0       0.0  0.0       0.0         0.0   
2       0.0      0.0  0.0         0.0       0.0  0.0       0.0         0.0   

   además 

In [42]:
print(tfidf_dfLema.shape[1])

5038


In [43]:
tfidf_dfStem.head(3)

aaahm  aah  aahhh  aandaluci  abac  abandon  abandonadisim  abast  abces  \
0    0.0  0.0    0.0        0.0   0.0      0.0            0.0    0.0    0.0   
1    0.0  0.0    0.0        0.0   0.0      0.0            0.0    0.0    0.0   
2    0.0  0.0    0.0        0.0   0.0      0.0            0.0    0.0    0.0   

   abedul  abel  abiert  abog  abracit  abraz  abrazart  abrazot  abrazotee  \
0     0.0   0.0     0.0   0.0      0.0    0.0       0.0      0.0        0.0   
1     0.0   0.0     0.0   0.0      0.0    0.0       0.0      0.0        0.0   
2     0.0   0.0     0.0   0.0      0.0    0.0       0.0      0.0        0.0   

   abre  abrig  abrir  abrist  abrum  abs  absolut  abstract  absurd  abu  \
0   0.0    0.0    0.0     0.0    0.0  0.0      0.0       0.0     0.0  0.0   
1   0.0    0.0    0.0     0.0    0.0  0.0      0.0       0.0     0.0  0.0   
2   0.0    0.0    0.0     0.0    0.0  0.0      0.0       0.0     0.0  0.0   

   abuel  abuelit  aburr  aca  acab  acamp  acc  accederi  accident  accion  \
0    0.0      0.0    0.0  0.0   0.0    0.0  0.0       0.0       0.0     0.0   
1    0.0      0.0    0.0  0.0   0.0    0.0  0.0       0.0       0.0     0.0   
2    0.0      0.0    0.0  0.0   0.0    0.0  0.0       0.0       0.0     0.0   

   aceit  aceitun  acent  acept  acerc  acert  achuchart  aciert  aclar  \
0    0.0      0.0    0.0    0.0    0.0    0.0        0.0     0.0    0.0   
1    0.0      0.0    0.0    0.0    0.0    0.0        0.0     0.0    0.0   
2    0.0      0.0    0.0    0.0    0.0    0.0        0.0     0.0    0.0   

   acomod  acompañ  acompañart  acondion  aconsej  acos  acost  acostumbr  \
0     0.0      0.0         0.0       0.0      0.0   0.0    0.0        0.0   
1     0.0      0.0         0.0       0.0      0.0   0.0    0.0        0.0   
2     0.0      0.0         0.0       0.0      0.0   0.0    0.0        0.0   

   actitud  activ  acto  actor  actriz  actuacion  actualiz  acuarelastmr  \
0      0.0    0.0   0.0    0.0     0.0        0.0       0.0           0.0   
1      0.0    0.0   0.0    0.0     0.0        0.0       0.0           0.0   
2      0.0    0.0   0.0    0.0     0.0        0.0       0.0           0.0   

   acuerd  acuerdo  acuest  adapt  adc  adel  adelgaz  ademas  ademasl  \
0     0.0      0.0     0.0    0.0  0.0   0.0      0.0     0.0      0.0   
1     0.0      0.0     0.0    0.0  0.0   0.0      0.0     0.0      0.0   
2     0.0      0.0     0.0    0.0  0.0   0.0      0.0     0.0      0.0   

   adiccion  adiosagost  adivin  adjet  admir  adopt  ador  adqu  adriel  \
0       0.0         0.0     0.0    0.0    0.0    0.0   0.0   0.0     0.0   
1       0.0         0.0     0.0    0.0    0.0    0.0   0.0   0.0     0.0   
2       0.0         0.0     0.0    0.0    0.0    0.0   0.0   0.0     0.0   

   adueñ  adult  advert  advirt  afect  afganistan  aficion  afin  afor  \
0    0.0    0.0     0.0     0.0    0.0         0.0      0.0   0.0   0.0   
1    0.0    0.0     0.0     0.0    0.0         0.0      0.0   0.0   0.0   
2    0.0    0.0     0.0     0.0    0.0         0.0      0.0   0.0   0.0   

   afortun  again  agam  agarr  agenci  agend  agent  ago  agot  agrad  \
0      0.0    0.0   0.0    0.0     0.0    0.0    0.0  0.0   0.0    0.0   
1      0.0    0.0   0.0    0.0     0.0    0.0    0.0  0.0   0.0    0.0   
2      0.0    0.0   0.0    0.0     0.0    0.0    0.0  0.0   0.0    0.0   

   agradec  agradecert  agradezc  agre  agreg  agres  agresion  agu  aguacer  \
0      0.0         0.0       0.0   0.0    0.0    0.0       0.0  0.0      0.0   
1      0.0         0.0       0.0   0.0    0.0    0.0       0.0  0.0      0.0   
2      0.0         0.0       0.0   0.0    0.0    0.0       0.0  0.0      0.0   

   aguant  aguit  agust   ah  ahh  ahhh  ahi  ahilodej  ahno  ahor  ahorc  \
0     0.0    0.0    0.0  0.0  0.0   0.0  0.0       0.0   0.0   0.0    0.0   
1     0.0    0.0    0.0  0.0  0.0   0.0  0.0       0.0   0.0   0.0    0.0   
2     0.0    0.0    0.0  0.0  0.0   0.0  0.0       0.0   0.0   0.0    0.

In [45]:
print(tfidf_dfStem.shape[1])

4071


In [46]:
total_registros = tfidf_dfLema.shape[0]
print("Total de registros Lema:", total_registros)
total_registros = tfidf_dfStem.shape[0]
print("Total de registros Stemmer:", total_registros)

Total de registros Lema: 1839
Total de registros Stemmer: 1839


In [ ]:
#vaciar dataframes
#tfidf_dfLema = tfidf_dfLema.drop(index=tfidf_dfLema.index)
#tfidf_dfStem= tfidf_dfStem.drop(index=tfidf_dfStem.index)

In [47]:
#agregar columna de id a los 2 datasets tfidf
tfidf_dfLema['Indice']=tfidf_dfLema.index
tfidf_dfStem['Indice']=tfidf_dfStem.index

#agregar la columna de id al dataset que contiene las etiquetas
df_Spanish['Indice']=df_Spanish.index
df_Spanish['Trg']=df_Spanish['label']

In [48]:
df_Spanish.head(3)

,text,label,Indice,Trg
0,estoy hasta el ojete de que me digáis que tengo cara de mala leche,negative,0,negative
1,@user Por? Tenía pensado verla después de la segunda de Daredevil,neutral,1,neutral
2,Esto de estar feliz mola,positive,2,positive


In [49]:
#hacer join/merge con tfidf Lema y el dataset original que tiene las etiquetas
#Los campos en lema son en minusculas y el campo Indice con I mayuscula
tfidf_dfLema=pd.merge(tfidf_dfLema,df_Spanish[['Trg','Indice']], on='Indice')
tfidf_dfLema = tfidf_dfLema.drop('Indice', axis=1)
tfidf_dfLema.head(3)

aaahm  aah  aahhh  aandaluciar  abandona  abandonadisimo  abandonado  \
0    0.0  0.0    0.0          0.0       0.0             0.0         0.0   
1    0.0  0.0    0.0          0.0       0.0             0.0         0.0   
2    0.0  0.0    0.0          0.0       0.0             0.0         0.0   

   abandonar  abasto  abceso  abedul  abel  abierto  abogado  abracito  \
0        0.0     0.0     0.0     0.0   0.0      0.0      0.0       0.0   
1        0.0     0.0     0.0     0.0   0.0      0.0      0.0       0.0   
2        0.0     0.0     0.0     0.0   0.0      0.0      0.0       0.0   

   abrazarte  abrazo  abrazote  abrazoteee  abrigo  abrir  abristir  \
0        0.0     0.0       0.0         0.0     0.0    0.0       0.0   
1        0.0     0.0       0.0         0.0     0.0    0.0       0.0   
2        0.0     0.0       0.0         0.0     0.0    0.0       0.0   

   abrumador  abs  absolutamente  absoluto  abstracto  absurdo  abu  abuelito  \
0        0.0  0.0            0.0       0.0        0.0      0.0  0.0       0.0   
1        0.0  0.0            0.0       0.0        0.0      0.0  0.0       0.0   
2        0.0  0.0            0.0       0.0        0.0      0.0  0.0       0.0   

   abuelo  aburrida  aburrido  aburrimiento  aburrir  acaba  acabado  acabar  \
0     0.0       0.0       0.0           0.0      0.0    0.0      0.0     0.0   
1     0.0       0.0       0.0           0.0      0.0    0.0      0.0     0.0   
2     0.0       0.0       0.0           0.0      0.0    0.0      0.0     0.0   

   acabo  acabé  acampar  acc  accederia  accidente  acción  aceit  aceituna  \
0    0.0    0.0      0.0  0.0        0.0        0.0     0.0    0.0       0.0   
1    0.0    0.0      0.0  0.0        0.0        0.0     0.0    0.0       0.0   
2    0.0    0.0      0.0  0.0        0.0        0.0     0.0    0.0       0.0   

   acento  aceptado  aceptar  acerca  acercar  acertado  acertar  achucharte  \
0     0.0       0.0      0.0     0.0      0.0       0.0      0.0         0.0   
1     0.0       0.0      0.0     0.0      0.0       0.0      0.0         0.0   
2     0.0       0.0      0.0     0.0      0.0       0.0      0.0         0.0   

   acierto  aclarar  aclare  acomodado  acompañado  acompañar  acompañarte  \
0      0.0      0.0     0.0        0.0         0.0        0.0          0.0   
1      0.0      0.0     0.0        0.0         0.0        0.0          0.0   
2      0.0      0.0     0.0        0.0         0.0        0.0          0.0   

   acompaño  acondionado  aconsejar  acordar  acoso  acostado  acostumbrado  \
0       0.0          0.0        0.0      0.0    0.0       0.0           0.0   
1       0.0          0.0        0.0      0.0    0.0       0.0           0.0   
2       0.0          0.0        0.0      0.0    0.0       0.0           0.0   

   acostumbrar  acostumbrarme  acostumbré  actitud  activado  actividad  \
0          0.0            0.0         0.0      0.0       0.0        0.0   
1          0.0            0.0         0.0      0.0       0.0        0.0   
2          0.0            0.0         0.0      0.0       0.0        0.0   

   activista  activo  acto  actor  actriz  actuación  actualizacion  \
0        0.0     0.0   0.0    0.0     0.0        0.0            0.0   
1        0.0     0.0   0.0    0.0     0.0        0.0            0.0   
2        0.0     0.0   0.0    0.0     0.0        0.0            0.0   

   actualizado  actualizar  actualizo  acuarelastmr  acuerda  acuerdo  \
0          0.0         0.0        0.0           0.0      0.0      0.0   
1          0.0         0.0        0.0           0.0      0.0      0.0   
2          0.0         0.0        0.0           0.0      0.0      0.0   

   acuerdoo  acuesto  acá  adaptación  adaptado  adc  adelante  adelgazado  \
0       0.0      0.0  0.0         0.0       0.0  0.0       0.0         0.0   
1       0.0      0.0  0.0         0.0       0.0  0.0       0.0         0.0   
2       0.0      0.0  0.0         0.0       0.0  0.0       0.0         0.0   

   además 

In [50]:
#hacer join/merge con tfidf Stemmer y el dataset original que tiene las etiquetas
#Los campos en lema son en minusculas y el campo Indice con I mayuscula
tfidf_dfStem=pd.merge(tfidf_dfStem,df_Spanish[['Trg','Indice']], on='Indice')
tfidf_dfStem = tfidf_dfStem.drop('Indice', axis=1)
tfidf_dfStem.head(3)

aaahm  aah  aahhh  aandaluci  abac  abandon  abandonadisim  abast  abces  \
0    0.0  0.0    0.0        0.0   0.0      0.0            0.0    0.0    0.0   
1    0.0  0.0    0.0        0.0   0.0      0.0            0.0    0.0    0.0   
2    0.0  0.0    0.0        0.0   0.0      0.0            0.0    0.0    0.0   

   abedul  abel  abiert  abog  abracit  abraz  abrazart  abrazot  abrazotee  \
0     0.0   0.0     0.0   0.0      0.0    0.0       0.0      0.0        0.0   
1     0.0   0.0     0.0   0.0      0.0    0.0       0.0      0.0        0.0   
2     0.0   0.0     0.0   0.0      0.0    0.0       0.0      0.0        0.0   

   abre  abrig  abrir  abrist  abrum  abs  absolut  abstract  absurd  abu  \
0   0.0    0.0    0.0     0.0    0.0  0.0      0.0       0.0     0.0  0.0   
1   0.0    0.0    0.0     0.0    0.0  0.0      0.0       0.0     0.0  0.0   
2   0.0    0.0    0.0     0.0    0.0  0.0      0.0       0.0     0.0  0.0   

   abuel  abuelit  aburr  aca  acab  acamp  acc  accederi  accident  accion  \
0    0.0      0.0    0.0  0.0   0.0    0.0  0.0       0.0       0.0     0.0   
1    0.0      0.0    0.0  0.0   0.0    0.0  0.0       0.0       0.0     0.0   
2    0.0      0.0    0.0  0.0   0.0    0.0  0.0       0.0       0.0     0.0   

   aceit  aceitun  acent  acept  acerc  acert  achuchart  aciert  aclar  \
0    0.0      0.0    0.0    0.0    0.0    0.0        0.0     0.0    0.0   
1    0.0      0.0    0.0    0.0    0.0    0.0        0.0     0.0    0.0   
2    0.0      0.0    0.0    0.0    0.0    0.0        0.0     0.0    0.0   

   acomod  acompañ  acompañart  acondion  aconsej  acos  acost  acostumbr  \
0     0.0      0.0         0.0       0.0      0.0   0.0    0.0        0.0   
1     0.0      0.0         0.0       0.0      0.0   0.0    0.0        0.0   
2     0.0      0.0         0.0       0.0      0.0   0.0    0.0        0.0   

   actitud  activ  acto  actor  actriz  actuacion  actualiz  acuarelastmr  \
0      0.0    0.0   0.0    0.0     0.0        0.0       0.0           0.0   
1      0.0    0.0   0.0    0.0     0.0        0.0       0.0           0.0   
2      0.0    0.0   0.0    0.0     0.0        0.0       0.0           0.0   

   acuerd  acuerdo  acuest  adapt  adc  adel  adelgaz  ademas  ademasl  \
0     0.0      0.0     0.0    0.0  0.0   0.0      0.0     0.0      0.0   
1     0.0      0.0     0.0    0.0  0.0   0.0      0.0     0.0      0.0   
2     0.0      0.0     0.0    0.0  0.0   0.0      0.0     0.0      0.0   

   adiccion  adiosagost  adivin  adjet  admir  adopt  ador  adqu  adriel  \
0       0.0         0.0     0.0    0.0    0.0    0.0   0.0   0.0     0.0   
1       0.0         0.0     0.0    0.0    0.0    0.0   0.0   0.0     0.0   
2       0.0         0.0     0.0    0.0    0.0    0.0   0.0   0.0     0.0   

   adueñ  adult  advert  advirt  afect  afganistan  aficion  afin  afor  \
0    0.0    0.0     0.0     0.0    0.0         0.0      0.0   0.0   0.0   
1    0.0    0.0     0.0     0.0    0.0         0.0      0.0   0.0   0.0   
2    0.0    0.0     0.0     0.0    0.0         0.0      0.0   0.0   0.0   

   afortun  again  agam  agarr  agenci  agend  agent  ago  agot  agrad  \
0      0.0    0.0   0.0    0.0     0.0    0.0    0.0  0.0   0.0    0.0   
1      0.0    0.0   0.0    0.0     0.0    0.0    0.0  0.0   0.0    0.0   
2      0.0    0.0   0.0    0.0     0.0    0.0    0.0  0.0   0.0    0.0   

   agradec  agradecert  agradezc  agre  agreg  agres  agresion  agu  aguacer  \
0      0.0         0.0       0.0   0.0    0.0    0.0       0.0  0.0      0.0   
1      0.0         0.0       0.0   0.0    0.0    0.0       0.0  0.0      0.0   
2      0.0         0.0       0.0   0.0    0.0    0.0       0.0  0.0      0.0   

   aguant  aguit  agust   ah  ahh  ahhh  ahi  ahilodej  ahno  ahor  ahorc  \
0     0.0    0.0    0.0  0.0  0.0   0.0  0.0       0.0   0.0   0.0    0.0   
1     0.0    0.0    0.0  0.0  0.0   0.0  0.0       0.0   0.0   0.0    0.0   
2     0.0    0.0    0.0  0.0  0.0   0.0  0.0       0.0   0.0   0.0    0.

In [51]:
# crea el archivo Lema TF-IDF
tfidf_dfLema.to_csv('tfidf_dfLema.csv',index=False)
# crea el archivo Stemmer TF-IDF
tfidf_dfStem.to_csv('tfidf_dfStem.csv',index=False)